In [1]:
import pandas as pd
from sklearn import linear_model
import numpy as np
import operator

In [3]:
alunos = pd.read_csv("../alunosUFCGAnon.csv")
pre_requisitos = pd.read_csv("../preRequisitos.csv")
periodo_disc = pd.read_csv("../periodoDisciplinas.csv")

In [10]:
# ajustando data frame que servirá de entrada
alunos_cc = alunos.query("Cod_Curso == 14102100 & Tipo == 'Obrigatória' ")
# alunos_cc.sort_values('Media_Disciplina', ascending=False).drop_duplicates(['Nome_Disciplina','Matricula'])
COMPUTACAOtdf = pd.pivot_table(alunos_cc, values = 'Media_Disciplina', index = ['Nome_Disciplina'], columns = 'Matricula')
notas_alunos = alunos_cc
notas_alunos['Matricula'] = alunos_cc['Matricula'].map(lambda x: x.lstrip('B'))
notas_alunos = pd.pivot_table(notas_alunos, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# cadeiras que não exigem prerrequisito
sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                    'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                     'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                       'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']


/home/livia.cavalcanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
notas_aprovados = alunos_cc.loc[alunos_cc.Situacao == 'Aprovado']
notas_aprovados['Matricula'] = notas_aprovados['Matricula'].map(lambda x: x.lstrip('B'))
notas_aprovados = pd.pivot_table(notas_aprovados, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')

/home/livia.cavalcanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
%run util.ipynb
corr_disciplinas = notas_alunos.corr() 

In [6]:
# guardandos as regressoes para cada disciplina em um dicionario
disciplinas_regressoes = {}
regressoes = {}
regressao = linear_model.LinearRegression()
for disciplina in periodo_disc["DISCIPLINA"]:
    # pegando o periodo da disciplina da iteração
    periodo = periodo_disc.loc[periodo_disc["DISCIPLINA"] == disciplina]['PERIODO'].values[0]
    # pegando as disciplinas que são do mesmo periodo ou anteriores a disciplina em questão
    disciplinas_regressao = periodo_disc.loc[periodo_disc["PERIODO"]<=periodo]["DISCIPLINA"].values
    disciplinas_regressao = np.setdiff1d(corr_disciplinas.loc[disciplinas_regressao].sort_values(by=disciplina,ascending=False).drop([disciplina])[[disciplina]][:5].index, disciplina)
    disciplinas_regressoes[disciplina] = disciplinas_regressao
    disciplinas_regressao = notas_alunos[np.append(disciplinas_regressao,disciplina)].dropna(axis=0,how="any")

    X = disciplinas_regressao.loc[:,disciplinas_regressao.columns != disciplina]
    y = disciplinas_regressao.loc[:,disciplinas_regressao.columns == disciplina]
    regressoes[disciplina] = regressao.fit(X,y)



In [28]:
alunos_predicao = notas_aprovados.sample(n=5)
for index, aluno in alunos_predicao.iterrows():
    sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                        'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                         'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                           'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']
    
    # coletando quais cadeiras o aluno já foi aprovado
    cadeiras_aluno = aluno
    cadeiras_pagas = cadeiras_aluno.dropna().to_frame()
#     cadeiras_pagas = cadeiras_pagas[cadeiras_pagas.iloc[:,0] >= 5]

    cadeira_possivel = []

    # vendo quais cadeiras sem prerrequisito já foram pagas pelo aluno
    for cadeira in sem_prerequisito:
        if not cadeira in cadeiras_pagas.index:
            cadeira_possivel.append(cadeira)
    prox_possiveis_cadeiras = prox_cadeiras_nome(cadeiras_pagas, cadeira_possivel, pre_requisitos)
    
    cadeiras_aluno = cadeiras_aluno.fillna(0)
    dict_notas = {}
    for cadeira in prox_possiveis_cadeiras:

        dict_notas[cadeira]=(regressoes[cadeira].predict(np.array(cadeiras_aluno[disciplinas_regressoes[cadeira]].fillna(0).values.reshape(1,-1))))
        
 
    
    
    # imprimindo notas preditas
    list_notas = sorted(dict_notas.items(), key = operator.itemgetter(1), reverse = True)
    print("Notas do historico do aluno:")
    print(aluno.T.dropna())
    print("-----------")
    print("Notas preditas: ")
    if not list_notas:
        print("Sem recomendaçao")
    else:
        for cadeira in list_notas: 
            print(cadeira[0], ": ", cadeira[1][0][0])
    
    print("================ \n")


Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          8.6
CALCULO DIFERENCIAL E INTEGRAL II         5.8
FUNDAMENTOS DE FÍSICA CLÁSSICA            7.7
INTRODUÇÃO A COMPUTAÇÃO                   9.4
LABORATÓRIO DE PROGRAMAÇÃO I              8.1
LABORATÓRIO DE PROGRAMAÇÃO II             8.7
MATEMÁTICA DISCRETA                       7.6
PROGRAMAÇÃO I                             8.0
PROGRAMAÇÃO II                            8.7
TEORIA DOS GRAFOS                         8.4
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    8.5
Name: 10940, dtype: float64
-----------
Notas preditas: 
LEITURA E PRODUCAO DE TEXTOS :  9.01201088405
GERÊNCIA DA INFORMAÇÃO :  8.99936363057
METODOLOGIA CIENTÍFICA :  8.99936363057
ALGEBRA LINEAR I :  7.91810473044
INFORMÁTICA E SOCIEDADE :  7.57360631105
TEORIA DA COMPUTAÇÃO :  4.48639412745
ESTRUTURA DE DADOS E ALGORITMOS :  4.1553904148
FUNDAMENTOS DE FÍSICA MODERNA :  1.12880268182
LAB.DE ESTRUTURA DE DADOS E ALGORITMOS :  1.12880268182
PR